## 自定义输出解析器

在某些情况下，您可能希望实现自定义解析器以将模型输出构造为自定义格式。

有两种方法可以实现自定义解析器：
- 在 LCEL 中使用 RunnableLambda 或 RunnableGenerator – 我们强烈建议大多数用例使用此方法
- 通过从基类之一继承进行解析——这是困难方法

这两种方法之间的差异大多是表面的，主要在于触发哪些回调（例如， on_chain_start 与 on_parser_start ）

### 可运行的 Lambda 和生成器

推荐的解析方法是使用可运行的 lambda 和可运行的生成器！

在这里，我们将进行一个简单的解析，反转模型输出的大小写。

例如，如果模型输出：“Meow”，解析器将生成“mEOW”。


In [1]:
from typing import Iterable
from langchain_core.messages import AIMessage, AIMessageChunk

from langchain_openai import ChatOpenAI, OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.3,
)


def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()


chain = model | parse
chain.invoke("Hello")

'hELLO! hOW CAN i HELP YOU TODAY?'

In [13]:
for chunk in chain.stream("tell me a story"):
    print(chunk, end="")

oNCE UPON A TIME, IN THE HEART OF A BUSTLING CITY, THERE LIVED A YOUNG BOY NAMED jACK. dESPITE LIVING IN A FAST-PACED ENVIRONMENT FILLED WITH NOISE AND DISTRACTIONS, jACK HAD AN INSATIABLE CURIOSITY FOR NATURE AND WILDLIFE. hE WOULD OFTEN SPEND HIS WEEKENDS EXPLORING NEARBY PARKS AND FORESTS, OBSERVING BIRDS, INSECTS, AND OTHER CREATURES THAT CAUGHT HIS ATTENTION.

oNE DAY, WHILE WANDERING THROUGH THE CITY'S LARGEST PARK, jACK STUMBLED UPON A SMALL, HIDDEN POND SURROUNDED BY LUSH GREENERY. iT WAS A SERENE OASIS IN THE MIDDLE OF THE CONCRETE JUNGLE, AND IT IMMEDIATELY CAPTURED HIS INTEREST. aS HE APPROACHED THE WATER'S EDGE, HE NOTICED SOMETHING UNUSUAL: A TINY FROG, NO BIGGER THAN A COIN, SITTING ON A LILY PAD. tHE LITTLE CREATURE SEEMED TO BE LOOKING BACK AT jACK WITH WIDE, CURIOUS EYES.

jACK WAS FASCINATED BY THIS SMALL AMPHIBIAN AND DECIDED TO SIT DOWN NEARBY, OBSERVING IT IN SILENCE. hE WATCHED AS THE FROG LEAPED FROM ONE LILY PAD TO ANOTHER, ITS WEBBED FEET PROPELLING IT EFFORTLE

In [9]:
from langchain_core.runnables import RunnableGenerator


def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.content.swapcase()


streaming_parse = RunnableGenerator(streaming_parse)

In [16]:
chain = model | streaming_parse
for chunk in chain.stream("tell me a story"):
    print(chunk, end="")

oNCE UPON A TIME IN THE HEART OF aFRICA, THERE WAS A SMALL VILLAGE CALLED kIBURI. tHE VILLAGERS WERE SIMPLE PEOPLE WHO LIVED OFF THE LAND AND RELIED ON THEIR CROPS FOR SUSTENANCE. oNE DAY, A TERRIBLE DROUGHT STRUCK THE REGION, LEAVING THE FIELDS BARREN AND THE VILLAGERS DESPERATE FOR FOOD.

tHE CHIEF OF kIBURI, mWAMBA, SUMMONED ALL THE ELDERS TO DISCUSS WHAT THEY SHOULD DO IN THIS DIRE SITUATION. aFTER MUCH DELIBERATION, IT WAS DECIDED THAT THEY WOULD SEND OUT A GROUP OF YOUNG MEN TO SEARCH FOR HELP FROM NEIGHBORING VILLAGES.

aMONG THESE BRAVE YOUNG MEN WAS mWAMBA'S SON, kAMAU. hE WAS DETERMINED TO FIND FOOD AND SAVE HIS VILLAGE. tHE JOURNEY WAS LONG AND ARDUOUS, FILLED WITH MANY CHALLENGES AND OBSTACLES. bUT THE YOUNG MEN PERSEVERED, DRIVEN BY THEIR DESIRE TO HELP THEIR FAMILIES AND FRIENDS BACK IN kIBURI.

aFTER WEEKS OF SEARCHING, THEY FINALLY FOUND A VILLAGE THAT WAS WILLING TO SHARE ITS RESOURCES WITH THEM. wITH GRATITUDE IN THEIR HEARTS, THE YOUNG MEN RETURNED HOME WITH SACKS FU